In [1]:
import pickle
import numpy as np
import pandas as pd
import statistics
from os import listdir
import pycaret
from pycaret.classification import *
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

## SWELL

In [2]:
swell = pd.read_csv("Final_CSVs/swell.csv")

In [3]:
swell = swell.drop(columns=["Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "timestamp", "C"])

In [4]:
swell['Condition'] = np.where(swell['Condition']=='N',0,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='R',0,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='I',1,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='T',1,swell['Condition'])

## User-based Models

In [5]:
unique_participants4 = swell["PP"].unique()

In [6]:
len(unique_participants4)

25

In [7]:
print(unique_participants4)

['PP1' 'PP2' 'PP3' 'PP4' 'PP5' 'PP6' 'PP7' 'PP8' 'PP9' 'PP10' 'PP11'
 'PP12' 'PP13' 'PP14' 'PP15' 'PP16' 'PP17' 'PP18' 'PP19' 'PP20' 'PP21'
 'PP22' 'PP23' 'PP24' 'PP25']


In [8]:
person_group4 = swell.groupby('PP')

In [72]:
from sklearn.model_selection import train_test_split
import os  # Import os module for folder creation

# Specify folder name
folder_name = "SWELL_User_Based_Splitting_Output_Files"

# Load the CSV data into a pandas dataframe
df_user_info = pd.read_csv("Scored_Surveys/swell_person.csv", sep=";")

accuracies_swell = []
precision_swell = []
recall_swell = []
f1scores_swell = []

for participant in unique_participants4:
    print("Participant: ",participant)    
    part_df = person_group4.get_group(participant)

    # -----------------------------------------------------

    train_data, test_data = train_test_split(part_df)

    fold_groups = train_data.PP

    # Save the 'id' column from the test set
    test_ids = test_data['PP']

    train_data = train_data.drop(columns=['PP'])
    test_data = test_data.drop(columns=['PP'])

    # -----------------------------------------------------

    # grid = setup(data=train_data, target='Condition', fix_imbalance = True, html=False, verbose=False, fold_strategy='groupkfold', fold=3, fold_groups=fold_groups, test_data=test_data) #fix_imbalance = True,
    grid = setup(data=train_data, target='Condition', fix_imbalance = True, html=False, verbose=False, test_data=test_data) #fix_imbalance = True,
    best = compare_models(sort='F1')
    accuracies_swell.append(pull()['Accuracy'][0])
    precision_swell.append(pull()['Prec.'][0])
    recall_swell.append(pull()['Recall'][0])
    f1scores_swell.append(pull()['F1'][0])
    print(best)

    # ---------------------------------------------------

    # Make predictions using the best model
    predictions = predict_model(best, data=test_data)

    # Add 'id' column back to predictions DataFrame
    predictions['PP'] = test_ids  # Use the 'id' column from the test data

    # Merge the dataframes based on the 'PP' column
    predictions_with_info = predictions.merge(df_user_info, on="PP", how="left")

    # Filter data for the current participant
    # participant_data = all_data[all_data['PP'] == participant]

    # Extract true labels (y_true), rename the column
    y_true = predictions_with_info[['Condition']].rename(columns={'Condition': 'y_true'})

    # Extract predicted labels (y_pred), rename the column
    y_pred = predictions_with_info[['prediction_label']].rename(columns={'prediction_label': 'y_pred'})

    # Identify protected attribute columns (assuming you know the column names)
    protected_attributes = predictions_with_info[['Age', 'Gender', 'Occupation']]

    # Concatenate DataFrames containing predictions and protected attributes
    all_data = pd.concat([test_ids.reset_index(drop=True), y_true, y_pred, protected_attributes], axis=1)

    # Create filename
    filename = f"predictions_{participant}.csv"

    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)  # Create folder using makedirs()

    # Create full path with folder name
    filepath = os.path.join(folder_name, filename)

    # Save predictions
    all_data.to_csv(filepath, index=False)

    print(f"Predictions saved to: {filepath}")

Participant:  PP1


                                    Model  Accuracy     AUC  Recall   Prec.  \
et                 Extra Trees Classifier    0.6800  0.6895   0.670  0.6983   
svm                   SVM - Linear Kernel    0.5289  0.0000   0.920  0.5500   
dt               Decision Tree Classifier    0.6456  0.6425   0.665  0.6450   
lightgbm  Light Gradient Boosting Machine    0.6478  0.7305   0.625  0.6738   
knn                K Neighbors Classifier    0.6789  0.7510   0.620  0.6571   
lda          Linear Discriminant Analysis    0.6367  0.6970   0.605  0.6663   
rf               Random Forest Classifier    0.6267  0.6895   0.625  0.6267   
gbc          Gradient Boosting Classifier    0.5856  0.7155   0.625  0.5788   
ridge                    Ridge Classifier    0.6156  0.0000   0.580  0.6321   
ada                  Ada Boost Classifier    0.5622  0.6425   0.575  0.5671   
lr                    Logistic Regression    0.5644  0.6620   0.540  0.5683   
qda       Quadratic Discriminant Analysis    0.5222 

                                    Model  Accuracy    AUC  Recall   Prec.  \
lightgbm  Light Gradient Boosting Machine    0.9222  0.990   0.930  0.9183   
rf               Random Forest Classifier    0.9111  0.980   0.930  0.9100   
ada                  Ada Boost Classifier    0.9111  0.975   0.905  0.9300   
gbc          Gradient Boosting Classifier    0.9111  0.960   0.905  0.9300   
et                 Extra Trees Classifier    0.9111  0.975   0.910  0.9300   
ridge                    Ridge Classifier    0.8889  0.000   0.975  0.8364   
lda          Linear Discriminant Analysis    0.8889  0.900   0.975  0.8364   
dt               Decision Tree Classifier    0.9000  0.900   0.885  0.9183   
knn                K Neighbors Classifier    0.8889  0.955   0.930  0.8600   
qda       Quadratic Discriminant Analysis    0.8333  0.910   0.930  0.7902   
lr                    Logistic Regression    0.8556  0.895   0.860  0.8664   
nb                            Naive Bayes    0.8222  0.865   0.9

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.7200  0.8281  0.7667  0.7633   
ada                  Ada Boost Classifier    0.7344  0.8115  0.6967  0.8145   
gbc          Gradient Boosting Classifier    0.6711  0.7927  0.6767  0.7345   
rf               Random Forest Classifier    0.6600  0.7805  0.6967  0.6995   
et                 Extra Trees Classifier    0.6600  0.7413  0.6967  0.7045   
dt               Decision Tree Classifier    0.6589  0.6819  0.6767  0.7212   
lightgbm  Light Gradient Boosting Machine    0.6989  0.7413  0.5933  0.7983   
svm                   SVM - Linear Kernel    0.5611  0.0000  0.5833  0.7111   
ridge                    Ridge Classifier    0.6656  0.0000  0.3833  1.0000   
lda          Linear Discriminant Analysis    0.6656  0.5781  0.3833  1.0000   
qda       Quadratic Discriminant Analysis    0.6444  0.4894  0.3833  0.9000   
lr                    Logistic Regression    0.6444 

                                    Model  Accuracy     AUC  Recall   Prec.  \
dt               Decision Tree Classifier    0.8556  0.8386  0.8900  0.8581   
lightgbm  Light Gradient Boosting Machine    0.8556  0.8829  0.8933  0.8514   
ada                  Ada Boost Classifier    0.8456  0.8632  0.8900  0.8414   
et                 Extra Trees Classifier    0.8444  0.8517  0.8700  0.8642   
knn                K Neighbors Classifier    0.8444  0.9011  0.8633  0.8579   
gbc          Gradient Boosting Classifier    0.8456  0.8712  0.8700  0.8531   
rf               Random Forest Classifier    0.8344  0.8698  0.8500  0.8608   
nb                            Naive Bayes    0.6778  0.8486  0.9400  0.6644   
qda       Quadratic Discriminant Analysis    0.6144  0.8416  0.9400  0.5916   
lr                    Logistic Regression    0.7067  0.8507  0.5967  0.8517   
ridge                    Ridge Classifier    0.7067  0.0000  0.5967  0.8517   
lda          Linear Discriminant Analysis    0.7067 

...

                                    Model  Accuracy     AUC  Recall   Prec.  \
et                 Extra Trees Classifier    0.6232  0.5788  0.6250  0.6595   
ada                  Ada Boost Classifier    0.5982  0.5898  0.5667  0.5917   
svm                   SVM - Linear Kernel    0.4429  0.0000  0.9000  0.4054   
gbc          Gradient Boosting Classifier    0.5607  0.5688  0.5500  0.5595   
rf               Random Forest Classifier    0.5857  0.6429  0.5250  0.6708   
dt               Decision Tree Classifier    0.4589  0.4683  0.5167  0.4258   
knn                K Neighbors Classifier    0.5125  0.5227  0.4500  0.4683   
lr                    Logistic Regression    0.6089  0.6658  0.4000  0.5433   
lda          Linear Discriminant Analysis    0.5750  0.6279  0.3750  0.4833   
qda       Quadratic Discriminant Analysis    0.6214  0.6075  0.3500  0.5167   
ridge                    Ridge Classifier    0.5857  0.0000  0.3750  0.4750   
lightgbm  Light Gradient Boosting Machine    0.4964 

                                    Model  Accuracy     AUC  Recall   Prec.  \
et                 Extra Trees Classifier    0.8711  0.8821  0.8467  0.9267   
nb                            Naive Bayes    0.8278  0.9195  1.0000  0.7798   
dt               Decision Tree Classifier    0.8389  0.8383  0.8267  0.8898   
rf               Random Forest Classifier    0.8300  0.9417  0.8133  0.8948   
knn                K Neighbors Classifier    0.8189  0.9192  0.8500  0.8483   
gbc          Gradient Boosting Classifier    0.8211  0.9360  0.7733  0.9148   
ridge                    Ridge Classifier    0.7867  0.0000  0.8600  0.8298   
ada                  Ada Boost Classifier    0.8089  0.9313  0.7533  0.9033   
lr                    Logistic Regression    0.7744  0.9358  0.7900  0.8490   
lda          Linear Discriminant Analysis    0.7556  0.9358  0.7767  0.8431   
lightgbm  Light Gradient Boosting Machine    0.7533  0.8992  0.7333  0.8250   
svm                   SVM - Linear Kernel    0.7667 

                                    Model  Accuracy     AUC  Recall   Prec.  \
lr                    Logistic Regression    0.7389  0.7465   0.635  0.7550   
nb                            Naive Bayes    0.5311  0.7285   1.000  0.5222   
lightgbm  Light Gradient Boosting Machine    0.7289  0.7445   0.615  0.7550   
qda       Quadratic Discriminant Analysis    0.5111  0.0000   1.000  0.5111   
ridge                    Ridge Classifier    0.7267  0.0000   0.595  0.7800   
lda          Linear Discriminant Analysis    0.7267  0.7735   0.595  0.7800   
knn                K Neighbors Classifier    0.5278  0.6525   0.900  0.5540   
rf               Random Forest Classifier    0.7067  0.7145   0.575  0.7550   
et                 Extra Trees Classifier    0.7067  0.7110   0.575  0.7550   
gbc          Gradient Boosting Classifier    0.6856  0.6450   0.535  0.7500   
ada                  Ada Boost Classifier    0.6644  0.6370   0.490  0.7333   
dt               Decision Tree Classifier    0.6544 

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.9167  0.9405    1.00  0.8714   
rf               Random Forest Classifier    0.9178  0.9440    0.98  0.8833   
gbc          Gradient Boosting Classifier    0.9178  0.9450    0.98  0.8833   
dt               Decision Tree Classifier    0.9067  0.9110    0.98  0.8667   
lightgbm  Light Gradient Boosting Machine    0.8956  0.9240    0.98  0.8548   
et                 Extra Trees Classifier    0.8844  0.9500    0.96  0.8625   
ada                  Ada Boost Classifier    0.8756  0.9190    0.96  0.8381   
qda       Quadratic Discriminant Analysis    0.8144  0.7420    0.80  0.8433   
ridge                    Ridge Classifier    0.8044  0.0000    0.80  0.8283   
lda          Linear Discriminant Analysis    0.8044  0.7490    0.80  0.8283   
lr                    Logistic Regression    0.7844  0.7370    0.80  0.8033   
nb                            Naive Bayes    0.7422 

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.9178  0.9438  0.9467  0.9133   
rf               Random Forest Classifier    0.8956  0.9246  0.9067  0.9148   
ada                  Ada Boost Classifier    0.8844  0.9042  0.9067  0.8981   
lightgbm  Light Gradient Boosting Machine    0.8667  0.8876  0.9433  0.8371   
gbc          Gradient Boosting Classifier    0.8744  0.8664  0.8900  0.8981   
et                 Extra Trees Classifier    0.8622  0.8895  0.8667  0.8898   
dt               Decision Tree Classifier    0.8522  0.8310  0.8500  0.8898   
nb                            Naive Bayes    0.6967  0.9203  1.0000  0.6600   
qda       Quadratic Discriminant Analysis    0.5522  0.0000  1.0000  0.5522   
svm                   SVM - Linear Kernel    0.6978  0.0000  0.6633  0.8606   
lr                    Logistic Regression    0.7178  0.9203  0.5433  0.9300   
ridge                    Ridge Classifier    0.7178 

In [73]:
mean_acc_swell = statistics.mean(accuracies_swell)
mean_prec_swell = statistics.mean(precision_swell)
mean_rec_swell = statistics.mean(recall_swell)
mean_f1_swell = statistics.mean(f1scores_swell)
std_acc_swell = statistics.stdev(accuracies_swell)
std_prec_swell = statistics.stdev(precision_swell)
std_rec_swell = statistics.stdev(recall_swell)
std_f1_swell = statistics.stdev(f1scores_swell)

In [74]:
print("Mean Accuracy SWELL: ",mean_acc_swell)
print("Mean Precision SWELL: ",mean_prec_swell)
print("Mean Recall SWELL: ",mean_rec_swell)
print("Mean F1-score SWELL: ",mean_f1_swell)
print("Standard deviation of Accuracy SWELL: ",std_acc_swell)
print("Standard deviation of Precision SWELL: ",std_prec_swell)
print("Standard deviation of Recall SWELL: ",std_rec_swell)
print("Standard deviation of F1-score SWELL: ",std_f1_swell)

Mean Accuracy SWELL:  0.805528
Mean Precision SWELL:  0.809288
Mean Recall SWELL:  0.849868
Mean F1-score SWELL:  0.816344
Standard deviation of Accuracy SWELL:  0.10705632676306431
Standard deviation of Precision SWELL:  0.10372032234170248
Standard deviation of Recall SWELL:  0.12015854013205497
Standard deviation of F1-score SWELL:  0.1092197405844444


In [75]:
folder_path = "SWELL_User_Based_Splitting_Output_Files"

# List all CSV files in the folder:
csv_files = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        csv_files.append(os.path.join(folder_path, filename))

# Concatenate files
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

In [76]:
output_folder = "Output_Files"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder using makedirs()

# Create the full path with the output folder name
output_filepath = os.path.join(output_folder, "SWELL_User_Based_Splitting.csv")

# Save the concatenated DataFrame to the new CSV file
combined_df.to_csv(output_filepath, index=False)

print(f"All predictions saved to: {output_filepath}")

All predictions saved to: Output_Files\SWELL_User_Based_Splitting.csv


### User-Based Splitting model with protected attributes

In [77]:
swell = pd.read_csv("Final_CSVs/swell.csv")

In [78]:
swell = swell.drop(columns=["Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "timestamp", "C"])

In [79]:
swell['Condition'] = np.where(swell['Condition']=='N',0,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='R',0,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='I',1,swell['Condition'])
swell['Condition'] = np.where(swell['Condition']=='T',1,swell['Condition'])

In [80]:
# Load the CSV data into a pandas dataframe
df_user_info = pd.read_csv("Scored_Surveys/swell_person.csv", sep=";")

# Merge the dataframes based on the 'PP' column
swell_with_info = swell.merge(df_user_info, on="PP", how="left")

# Print the result
print(swell_with_info)

        PP Condition   HR       RMSSD         SCL  Age Gender  \
0      PP1         0  999  999.000000   80.239727   27      m   
1      PP1         0   61    0.061420   77.365127   27      m   
2      PP1         0   64    0.049663   77.359559   27      m   
3      PP1         0   60    0.052487   76.728772   27      m   
4      PP1         0   61    0.051189   76.512877   27      m   
...    ...       ...  ...         ...         ...  ...    ...   
3135  PP25         1  999  999.000000  999.000000   26      m   
3136  PP25         1  999  999.000000  999.000000   26      m   
3137  PP25         1  999  999.000000  999.000000   26      m   
3138  PP25         1  999  999.000000  999.000000   26      m   
3139  PP25         1  999  999.000000  999.000000   26      m   

                      Occupation Dominant hand Glasses  smoke  coffee  \
0                        student         right      no      6       6   
1                        student         right      no      6       6   


In [81]:
unique_participants4 = swell_with_info["PP"].unique()

In [82]:
person_group4 = swell_with_info.groupby('PP')

In [85]:
from sklearn.model_selection import train_test_split
import os  # Import os module for folder creation

# Specify folder name
folder_name = "SWELL_User_Based_Splitting_Output_Files_Bias"

accuracies_swell = []
precision_swell = []
recall_swell = []
f1scores_swell = []

for participant in unique_participants4:
    print("Participant: ",participant)    
    part_df = person_group4.get_group(participant)

    # -----------------------------------------------------

    train_data, test_data = train_test_split(part_df)

    fold_groups = train_data.PP

    # Save the 'id' column from the test set
    test_ids = test_data['PP']

    train_data = train_data.drop(columns=['PP'])
    test_data = test_data.drop(columns=['PP'])

    # -----------------------------------------------------

    grid = setup(data=train_data, target='Condition', fix_imbalance = True, html=False, verbose=False, test_data=test_data) #fix_imbalance = True,
    best = compare_models(sort='F1')
    accuracies_swell.append(pull()['Accuracy'][0])
    precision_swell.append(pull()['Prec.'][0])
    recall_swell.append(pull()['Recall'][0])
    f1scores_swell.append(pull()['F1'][0])
    print(best)

    # ---------------------------------------------------

    # Make predictions using the best model
    predictions = predict_model(best, data=test_data)

    # Add 'id' column back to predictions DataFrame
    predictions['PP'] = test_ids  # Use the 'id' column from the test data

    # Extract true labels (y_true), rename the column
    y_true = predictions[['Condition']].rename(columns={'Condition': 'y_true'})

    # Extract predicted labels (y_pred), rename the column
    y_pred = predictions[['prediction_label']].rename(columns={'prediction_label': 'y_pred'})

    # Identify protected attribute columns (assuming you know the column names)
    protected_attributes = predictions[['Age', 'Gender', 'Occupation']]

    # Concatenate DataFrames containing predictions and protected attributes
    all_data = pd.concat([test_ids, y_true, y_pred, protected_attributes], axis=1)

    # Create filename
    filename = f"predictions_{participant}.csv"

    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)  # Create folder using makedirs()

    # Create full path with folder name
    filepath = os.path.join(folder_name, filename)

    # Save predictions
    all_data.to_csv(filepath, index=False)

    print(f"Predictions saved to: {filepath}")

Participant:  PP1


                                    Model  Accuracy     AUC  Recall   Prec.  \
et                 Extra Trees Classifier    0.7378  0.7073  0.6867  0.8238   
rf               Random Forest Classifier    0.7056  0.6960  0.7067  0.7405   
gbc          Gradient Boosting Classifier    0.6989  0.7228  0.6867  0.7688   
knn                K Neighbors Classifier    0.6756  0.7457  0.6833  0.6948   
dt               Decision Tree Classifier    0.6667  0.6683  0.6467  0.7355   
ada                  Ada Boost Classifier    0.6378  0.6780  0.6100  0.6800   
lr                    Logistic Regression    0.6322  0.7070  0.6467  0.6743   
ridge                    Ridge Classifier    0.6222  0.0000  0.6667  0.6586   
lightgbm  Light Gradient Boosting Machine    0.6244  0.6725  0.6267  0.6650   
lda          Linear Discriminant Analysis    0.6122  0.7000  0.6267  0.6543   
svm                   SVM - Linear Kernel    0.5200  0.0000  0.5600  0.3939   
nb                            Naive Bayes    0.4578 

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.9222  0.9525   0.960  0.9067   
lda          Linear Discriminant Analysis    0.9111  0.9200   1.000  0.8695   
rf               Random Forest Classifier    0.9111  0.9450   0.930  0.9050   
ridge                    Ridge Classifier    0.8889  0.0000   0.955  0.8662   
lr                    Logistic Regression    0.9000  0.9250   0.905  0.9000   
et                 Extra Trees Classifier    0.9000  0.9400   0.910  0.9100   
nb                            Naive Bayes    0.8667  0.9025   0.980  0.8171   
ada                  Ada Boost Classifier    0.8889  0.9300   0.885  0.8967   
gbc          Gradient Boosting Classifier    0.8667  0.9250   0.840  0.8933   
lightgbm  Light Gradient Boosting Machine    0.8556  0.9550   0.815  0.8850   
dt               Decision Tree Classifier    0.8556  0.8500   0.815  0.8833   
svm                   SVM - Linear Kernel    0.6444 

                                    Model  Accuracy     AUC  Recall   Prec.  \
rf               Random Forest Classifier    0.7289  0.8449  0.7467  0.7889   
knn                K Neighbors Classifier    0.6989  0.8249  0.7833  0.7332   
ada                  Ada Boost Classifier    0.7100  0.8439  0.7133  0.7996   
lightgbm  Light Gradient Boosting Machine    0.7000  0.8546  0.6800  0.8290   
et                 Extra Trees Classifier    0.6578  0.7998  0.7133  0.7232   
gbc          Gradient Boosting Classifier    0.6667  0.8214  0.6900  0.7255   
dt               Decision Tree Classifier    0.6544  0.6954  0.6733  0.7421   
svm                   SVM - Linear Kernel    0.5533  0.0000  0.7200  0.5092   
lda          Linear Discriminant Analysis    0.5778  0.8851  0.2900  0.7833   
lr                    Logistic Regression    0.5556  0.8809  0.2300  0.8000   
ridge                    Ridge Classifier    0.5556  0.0000  0.2300  0.8000   
nb                            Naive Bayes    0.4933 

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.8544  0.9160  0.8800  0.8588   
lightgbm  Light Gradient Boosting Machine    0.8444  0.9050  0.8600  0.8588   
dt               Decision Tree Classifier    0.8333  0.8202  0.8433  0.8588   
gbc          Gradient Boosting Classifier    0.8233  0.8660  0.8233  0.8621   
ada                  Ada Boost Classifier    0.7900  0.8585  0.8233  0.7971   
et                 Extra Trees Classifier    0.7911  0.8264  0.8033  0.8088   
rf               Random Forest Classifier    0.7800  0.8787  0.8033  0.7888   
nb                            Naive Bayes    0.6800  0.8390  0.7067  0.7399   
svm                   SVM - Linear Kernel    0.6756  0.0000  0.5867  0.6955   
lr                    Logistic Regression    0.6778  0.8317  0.5467  0.7717   
ridge                    Ridge Classifier    0.6778  0.0000  0.5467  0.7717   
lda          Linear Discriminant Analysis    0.6778 

...

                                    Model  Accuracy     AUC  Recall   Prec.  \
ada                  Ada Boost Classifier    0.6839  0.7292  0.6750  0.6383   
rf               Random Forest Classifier    0.6571  0.6535  0.6167  0.6000   
gbc          Gradient Boosting Classifier    0.6554  0.6996  0.6083  0.6167   
knn                K Neighbors Classifier    0.6071  0.5973  0.6500  0.6129   
et                 Extra Trees Classifier    0.6161  0.6227  0.5500  0.5833   
svm                   SVM - Linear Kernel    0.5554  0.0000  0.6417  0.5845   
lightgbm  Light Gradient Boosting Machine    0.5571  0.5850  0.5417  0.4800   
lr                    Logistic Regression    0.6071  0.7092  0.4167  0.6167   
ridge                    Ridge Classifier    0.6071  0.0000  0.4167  0.6167   
lda          Linear Discriminant Analysis    0.6071  0.7292  0.4167  0.6167   
dt               Decision Tree Classifier    0.5554  0.5442  0.4833  0.4567   
nb                            Naive Bayes    0.6589 

                                    Model  Accuracy     AUC  Recall   Prec.  \
dt               Decision Tree Classifier    0.8922  0.8925    0.88  0.9314   
rf               Random Forest Classifier    0.8922  0.9455    0.88  0.9314   
et                 Extra Trees Classifier    0.8811  0.8895    0.88  0.9064   
gbc          Gradient Boosting Classifier    0.8822  0.9405    0.86  0.9314   
nb                            Naive Bayes    0.8378  0.9297    1.00  0.7869   
ada                  Ada Boost Classifier    0.8500  0.9140    0.80  0.9214   
lr                    Logistic Regression    0.8267  0.9508    0.80  0.9024   
lda          Linear Discriminant Analysis    0.8178  0.9508    0.78  0.9131   
knn                K Neighbors Classifier    0.7833  0.9078    0.78  0.8240   
svm                   SVM - Linear Kernel    0.7800  0.0000    0.82  0.8317   
ridge                    Ridge Classifier    0.7867  0.0000    0.76  0.8893   
lightgbm  Light Gradient Boosting Machine    0.7878 

                                    Model  Accuracy     AUC  Recall   Prec.  \
ridge                    Ridge Classifier    0.7667  0.0000  0.6900  0.8917   
lda          Linear Discriminant Analysis    0.7667  0.8213  0.6900  0.8917   
rf               Random Forest Classifier    0.7678  0.8121  0.6733  0.9167   
et                 Extra Trees Classifier    0.7678  0.8096  0.6733  0.9167   
lr                    Logistic Regression    0.7567  0.8147  0.6733  0.8917   
nb                            Naive Bayes    0.5922  0.7813  1.0000  0.5856   
lightgbm  Light Gradient Boosting Machine    0.7589  0.7958  0.6400  0.9300   
dt               Decision Tree Classifier    0.7500  0.7617  0.6067  0.9550   
knn                K Neighbors Classifier    0.7167  0.7917  0.6933  0.8417   
gbc          Gradient Boosting Classifier    0.7367  0.7854  0.6000  0.9217   
ada                  Ada Boost Classifier    0.7178  0.6938  0.5500  0.9550   
svm                   SVM - Linear Kernel    0.6244 

                                    Model  Accuracy     AUC  Recall   Prec.  \
rf               Random Forest Classifier    0.9578  0.9750  1.0000  0.9417   
gbc          Gradient Boosting Classifier    0.9578  0.9833  1.0000  0.9417   
dt               Decision Tree Classifier    0.9478  0.9667  1.0000  0.9274   
knn                K Neighbors Classifier    0.9278  0.9542  1.0000  0.9024   
et                 Extra Trees Classifier    0.9278  0.9750  0.9833  0.9095   
lightgbm  Light Gradient Boosting Machine    0.9078  0.9458  0.9833  0.8845   
ada                  Ada Boost Classifier    0.8967  0.9283  0.9433  0.8955   
lr                    Logistic Regression    0.7867  0.7222  0.7567  0.8764   
ridge                    Ridge Classifier    0.7867  0.0000  0.7567  0.8764   
lda          Linear Discriminant Analysis    0.7867  0.7213  0.7567  0.8764   
nb                            Naive Bayes    0.6956  0.8337  0.7067  0.7489   
svm                   SVM - Linear Kernel    0.5589 

                                    Model  Accuracy     AUC  Recall   Prec.  \
knn                K Neighbors Classifier    0.8878  0.9115   0.935  0.8681   
lightgbm  Light Gradient Boosting Machine    0.8678  0.8700   0.915  0.8606   
rf               Random Forest Classifier    0.8467  0.9050   0.850  0.8548   
dt               Decision Tree Classifier    0.8356  0.7870   0.850  0.8429   
ada                  Ada Boost Classifier    0.8356  0.8525   0.850  0.8429   
gbc          Gradient Boosting Classifier    0.8356  0.8460   0.850  0.8429   
et                 Extra Trees Classifier    0.8356  0.8740   0.850  0.8348   
nb                            Naive Bayes    0.6167  0.9200   1.000  0.5687   
lr                    Logistic Regression    0.7622  0.9200   0.555  0.9167   
ridge                    Ridge Classifier    0.7622  0.0000   0.555  0.9167   
lda          Linear Discriminant Analysis    0.7622  0.9200   0.555  0.9167   
svm                   SVM - Linear Kernel    0.6422 

In [86]:
mean_acc_swell = statistics.mean(accuracies_swell)
mean_prec_swell = statistics.mean(precision_swell)
mean_rec_swell = statistics.mean(recall_swell)
mean_f1_swell = statistics.mean(f1scores_swell)
std_acc_swell = statistics.stdev(accuracies_swell)
std_prec_swell = statistics.stdev(precision_swell)
std_rec_swell = statistics.stdev(recall_swell)
std_f1_swell = statistics.stdev(f1scores_swell)

In [87]:
print("Mean Accuracy SWELL: ",mean_acc_swell)
print("Mean Precision SWELL: ",mean_prec_swell)
print("Mean Recall SWELL: ",mean_rec_swell)
print("Mean F1-score SWELL: ",mean_f1_swell)
print("Standard deviation of Accuracy SWELL: ",std_acc_swell)
print("Standard deviation of Precision SWELL: ",std_prec_swell)
print("Standard deviation of Recall SWELL: ",std_rec_swell)
print("Standard deviation of F1-score SWELL: ",std_f1_swell)

Mean Accuracy SWELL:  0.816788
Mean Precision SWELL:  0.82596
Mean Recall SWELL:  0.856396
Mean F1-score SWELL:  0.8288719999999999
Standard deviation of Accuracy SWELL:  0.10761224496620571
Standard deviation of Precision SWELL:  0.10686676673940003
Standard deviation of Recall SWELL:  0.11587951889786219
Standard deviation of F1-score SWELL:  0.1050498672060084


In [88]:
folder_path = "SWELL_User_Based_Splitting_Output_Files_Bias"

# List all CSV files in the folder:
csv_files = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        csv_files.append(os.path.join(folder_path, filename))

# Concatenate files
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

In [89]:
output_folder = "Output_Files"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # Create folder using makedirs()

# Create the full path with the output folder name
output_filepath = os.path.join(output_folder, "SWELL_User_Based_Splitting_Bias.csv")

# Save the concatenated DataFrame to the new CSV file
combined_df.to_csv(output_filepath, index=False)

print(f"All predictions saved to: {output_filepath}")

All predictions saved to: Output_Files\SWELL_User_Based_Splitting_Bias.csv
